In [39]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.neighbors import NearestNeighbors
from sklearn.decomposition import PCA
import SteamVariables as sv
from sklearn.utils import shuffle
import sys
from nltk.sentiment import SentimentIntensityAnalyzer
from importlib import reload

reload(sv)

df = pd.read_csv(sv.CSV_PATH, nrows=8000000)
df = df.dropna()

# sys.stdout.reconfigure(encoding='utf-8')

languages = ["bulgarian", "croatian", "danish", "czech", "slovak", "slovenian", "slovak", "slovenian", 
"spanish", "estonian", "finnish", "french", "greek", "hungarian", "irish", "italian", 
"latvian", "lithuanian", "maltese", "dutch", "polish", "portuguese", "romanian", "swedish", "english", "brazilian"]

# Eliminar linhas com linguagem diferente das selecionadas
new_dt = df[df[sv.LANGUAGE].isin(languages)]

def sentiment(review):
    sid = SentimentIntensityAnalyzer()
    score = sid.polarity_scores(review)
    return score

df = shuffle(df,random_state=60)

print(df[sv.APP_NAME])

sample_dt = df.sample(n=50000)
df = pd.DataFrame(sample_dt)

df["review_score"] = df[sv.REVIEW].apply(lambda x: sentiment(x))

feature_columnsNew= [sv.AUTHOR_STEAMID,sv.APP_NAME, sv.REVIEW_SCORE]

new_dt = df[feature_columnsNew]
print(new_dt)

2304169                        Subnautica
1223864                       Garry's Mod
7829537                              Rust
6959876                          Among Us
1848438                  Dead by Daylight
                        ...              
3330285     The Binding of Isaac: Rebirth
3951195             Insurgency: Sandstorm
6026956    Tom Clancy's Rainbow Six Siege
5229898                   DEATH STRANDING
396453           The Witcher 3: Wild Hunt
Name: app_name, Length: 7976244, dtype: object
            author.steamid                app_name  \
2009902  76561198128923366        Dead by Daylight   
1274385  76561198251966442             Garry's Mod   
2631398  76561198066111175                Banished   
4025446  76561198042506916          Slay the Spire   
4417602  76561198095452963  Euro Truck Simulator 2   
...                    ...                     ...   
4237104  76561198882604678  Euro Truck Simulator 2   
7102650  76561198281971200                Among Us   
48426

In [47]:
from surprise import KNNBasic
from surprise import Dataset
from surprise import accuracy
from surprise import SVD
from surprise import Reader
from surprise.model_selection import train_test_split, cross_validate

# df[sv.REVIEW_SCORE] = df[sv.REVIEW_SCORE].apply(lambda x: x["compound"])

# Função para converter os valores
def converter_valor(valor):
    if valor < -0.5:
        return 1
    elif valor >= -0.5 and valor < 0:
        return 2
    elif valor >= 0 and valor < 0.4:
        return 3
    elif valor >= 0.4 and valor < 0.8:
        return 4
    elif valor >= 0.8:
        return 5
    else:
        return 0

# Aplicar a função na coluna e criar uma nova coluna com os valores convertidos
df['review_score'] = df['review_score'].apply(converter_valor)

feature_columnsNew= [sv.AUTHOR_STEAMID,sv.APP_NAME, "review_score"]
new_dt = df[feature_columnsNew]

reader = Reader(rating_scale=(0, 1))
print(new_dt)
data= Dataset.load_from_df(new_dt,reader=reader)
trainSet, testSet = train_test_split(data,test_size=0.2,random_state=60)
algo = SVD()
print(data)
algo.fit(trainSet)
predictions = algo.test(testSet)
accuracy.rmse(predictions)
accuracy.mae(predictions)
cross_validate(algo,data,measures=['RMSE','MAE'],cv = 5, verbose=True)


            author.steamid                app_name  review_score
2009902  76561198128923366        Dead by Daylight             5
1274385  76561198251966442             Garry's Mod             5
2631398  76561198066111175                Banished             5
4025446  76561198042506916          Slay the Spire             5
4417602  76561198095452963  Euro Truck Simulator 2             5
...                    ...                     ...           ...
4237104  76561198882604678  Euro Truck Simulator 2             5
7102650  76561198281971200                Among Us             5
4842675  76561198107278602              Foundation             5
1410003  76561198105226857             Garry's Mod             5
708257   76561198436844052                Portal 2             5

[50000 rows x 3 columns]
RMSE: 4.0000
MAE:  4.0000
Evaluating RMSE, MAE of algorithm SVD on 5 split(s).

                  Fold 1  Fold 2  Fold 3  Fold 4  Fold 5  Mean    Std     
RMSE (testset)    4.0000  4.0000  4.000

{'test_rmse': array([4., 4., 4., 4., 4.]),
 'test_mae': array([4., 4., 4., 4., 4.]),
 'fit_time': (0.8170907497406006,
  0.6998710632324219,
  0.6968142986297607,
  0.7599277496337891,
  0.9544832706451416),
 'test_time': (0.053192138671875,
  0.04449915885925293,
  0.04489922523498535,
  0.0605158805847168,
  0.06244945526123047)}

In [48]:
pred = algo.predict(uid=72,iid=322110)
print(pred)

user: 72         item: 322110     r_ui = None   est = 1.00   {'was_impossible': False}


In [51]:
import difflib
import random



def get_game_id(book_title, metadata):
    
    """
    Gets the game ID for a game title based on the closest match in the metadata dataframe.
    """
    
    existing_titles = list(metadata[sv.APP_NAME].values)
    closest_titles = difflib.get_close_matches(book_title, existing_titles)
    book_id = metadata[metadata[sv.APP_NAME] == closest_titles[0]][sv.APP_ID].values[0]
    return book_id

def get_game_info(book_id, metadata):
    
    """
    Returns some basic information about a book given the book id and the metadata dataframe.
    """
    
    book_info = metadata[metadata[sv.APP_ID] == book_id][[sv.APP_ID, sv.APP_NAME]]

    game_info = book_info["app_name"].values[0]
    
    return game_info

def predict_review(user_id, book_title, model, metadata):
    
    """
    Predicts the review (on a scale of 1-5) that a user would assign to a specific book. 
    """
    
    book_id = get_game_id(book_title, metadata)
    review_prediction = model.predict(uid=user_id, iid=book_id)
    return review_prediction.est

def generate_recommendation(user_id, model, metadata, thresh=0.15):
    jogos = []
    """
    Generates a book recommendation for a user based on a rating threshold. Only
    books with a predicted rating at or above the threshold will be recommended
    """
    
    lista_app = list(df['app_name'].unique())
    random.shuffle(lista_app)
    
    i = 0
    for book_title in lista_app:
        rating = predict_review(user_id, book_title, model, metadata)
        print(rating)        

        if rating >= thresh:
            book_id = get_game_id(book_title, metadata)
            jogos.append(get_game_info(book_id, metadata))
            i += 1
        if i == 3:
            return jogos
    return jogos


In [52]:

print(generate_recommendation(76561198120121300,algo,df))

['Pogostuck: Rage With Your Friends', 'Far Cry 5', "Sid Meier's Civilization VI", 'Nether', 'Foundation', 'NBA 2K21', 'DRAGON BALL FighterZ', 'Getting Over It with Bennett Foddy', 'Overcooked! 2', 'Stay Out', 'The Binding of Isaac: Rebirth', 'Shovel Knight: Treasure Trove', '20XX', 'Portal 2', 'Slay the Spire', 'Momodora: Reverie Under the Moonlight', 'A Way Out', 'Axiom Verge', 'X-Blades', 'Deus Ex: The Fall', 'A Short Hike', 'Beat Saber', 'Hollow Knight', 'Factorio', 'Euro Truck Simulator 2', 'NBA 2K19', 'Hunt Down The Freeman', 'Farming Simulator 19', 'Human: Fall Flat', 'Nova Drift', 'DEATH STRANDING', 'Pathfinder: Kingmaker', 'RollerCoaster Tycoon World', 'Counter-Strike: Source', 'ATLAS', 'Warhammer 40,000: Dawn of War III', 'Wallpaper Engine', 'Identity', 'Slipstream', 'Artifact', 'Total War: THREE KINGDOMS', 'Due Process', 'Subnautica', 'Into the Breach', 'Banished', 'Warhammer: Vermintide 2', 'GOD EATER 3', 'Age of Empires: Definitive Edition', 'Rust', 'Dead by Daylight', 'Bom